# Change jpg files creation times metadata
It is often that metadata for jpg files is wrong (e.g. the camera was not set correctly)

In [1]:
import os
from PIL import Image
from datetime import datetime
import piexif

In [75]:
def get_jpg_filenames(directory):
    list_of_jpg = []
    for (dirpath, dirnames, filenames) in os.walk(directory):
        list_of_jpg   += [os.path.join(dirpath, file) for file in filenames if (file[-3:] == 'jpg') or (file[-3:] == 'JPG')]
    return list_of_jpg

def get_creation_time(full_path):
    image = Image.open(full_path)
    exif = image.getexif()
    creation_date = exif.get(36867)
    return creation_date

def get_creation_times(list_of_jpg):
    list_of_creation_times = []
    for jpg in list_of_jpg:
        try:
            image = Image.open(jpg)
            exif = image.getexif()
            creation_time = exif.get(36867)
        except:
            creation_time = None
        list_of_creation_times.append(creation_time)
    return list_of_creation_times

def change_jpg_datetime(filename, year, month, day):
    full_path = os.path.join(directory, filename)
    exif_dict = piexif.load(full_path)
    exif_dict['Exif'] = { piexif.ExifIFD.DateTimeOriginal: datetime(year, month, day, 0, 0, 0).strftime("%Y:%m:%d %H:%M:%S") }
    exif_bytes = piexif.dump(exif_dict)
    piexif.insert(exif_bytes, filename)

def change_jpg_list_datetime(jpg_filenames_list, year, month, day):
    for filename in jpg_filenames_list:
        full_path = os.path.join(directory, filename)
        exif_dict = piexif.load(full_path)
        exif_dict['Exif'] = {piexif.ExifIFD.DateTimeOriginal: datetime(year, month, day, 0, 0, 0).strftime("%Y:%m:%d %H:%M:%S")}
        exif_bytes = piexif.dump(exif_dict)
        piexif.insert(exif_bytes, filename)
        
def creation_times_range(creation_times_list):
    try:
        year =  [int(x[0:4]) for x in creation_times_list if x != None]
        month =  [int(x[5:7]) for x in creation_times_list if x != None]
        day = [int(x[8:10]) for x in creation_times_list if x != None]
        print('Files from:',min(year),':',min(month),':',min(day), 'to:', max(year),':',max(month),':',max(day))
    except:
        print('Warning!!! Some files with unknown time')
        if None in creation_times_list:
            print('Warning!!! Some files with creation time == None in the directory')

In [ ]:
%%time
# Scan over immediate subfolders in the main directory folder and check creation times (period) within these subfolders

directory = 'D:\Photo'
list_subfolders_with_paths = [f.path for f in os.scandir(directory) if f.is_dir()]
print('Number of immediate subfolders: ', len(list_subfolders_with_paths))
print()
for idx, subdirectory in enumerate(list_subfolders_with_paths):
    print(idx)
    print(subdirectory)
    jpg_filenames_list = get_jpg_filenames(subdirectory)
    print(len(jpg_filenames_list), 'jpg files in subdirectory')
    creation_times_list = get_creation_times(jpg_filenames_list)
    creation_times_range(creation_times_list)
    print()

In [121]:
# Check creation times in one of the subfolders (folder):
directory = r'D:\Photo\2010.08.31-09.11 Майорка'
jpg_filenames_list = get_jpg_filenames(directory)

creation_times_list = get_creation_times(jpg_filenames_list)
creation_times_range(creation_times_list)

Files from: 2007 : 1 : 1 to: 2010 : 9 : 31


In [119]:
# Change creation times in the folder for all jpg files
# change_jpg_list_datetime(jpg_filenames_list, 2010, 9, 11)
# creation_times_list = get_creation_times(jpg_filenames_list)
# creation_times_range(creation_times_list)

Files from: 2010 : 12 : 31 to: 2010 : 12 : 31


In [56]:
# Change single file creation datetime
# filename = r'C:\Users\evgen\OneDrive - Imperial College London\ML\Face\Photo\4.jpg'
# change_jpg_datetime(filename, 2014, 3, 10)